In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import datetime
%matplotlib inline

In [65]:
import collections

In [28]:
import jieba

In [48]:
data = pd.read_excel('看門狗20160101-201809.xlsx', sheet_name='看門狗')

In [49]:
data.head()

,公司,事件日,序號,TCRI(年/月),事件強度,大事件類別,小事件類別,事件內容,Unnamed: 8
0,1101 台泥,2018-09-14,001A001,3(2018/06),0.0,I_產業前景,IF02_投資/併購/組織策略,1.本公司與土耳其SANKO Holding A.S.公司就其下水泥事業群之股權投資事宜簽署...,NaN
1,1101 台泥,2018-09-14,001A002,3(2018/06),0.0,I_產業前景,IF02_投資/併購/組織策略,2.有關具體交易條件，待後續雙方正式簽署拘束性合約時，將依法令規定公告相關事項。,NaN
2,1101 台泥,2018-08-21,001A001,3(2018/03),0.0,I_產業前景,IF01_增資/減資/融資發債,馬來西亞商POWER INFRASTRUCTURE HOLDINGS(MALAYSIA)SD...,NaN
3,1101 台泥,2018-08-21,001A002,3(2018/03),0.0,I_產業前景,IF01_增資/減資/融資發債,電力股份有限公司，經營火力發電廠等業務。,NaN
4,1101 台泥,2018-08-21,001B001,3(2018/03),0.0,I_產業前景,IF01_增資/減資/融資發債,【TCRI 觀點_20180827】,NaN


In [50]:
data = data.drop(['事件強度','Unnamed: 8'],axis=1)

In [52]:
data.head()

,公司,事件日,序號,TCRI(年/月),大事件類別,小事件類別,事件內容
0,1101 台泥,2018-09-14,001A001,3(2018/06),I_產業前景,IF02_投資/併購/組織策略,1.本公司與土耳其SANKO Holding A.S.公司就其下水泥事業群之股權投資事宜簽署...
1,1101 台泥,2018-09-14,001A002,3(2018/06),I_產業前景,IF02_投資/併購/組織策略,2.有關具體交易條件，待後續雙方正式簽署拘束性合約時，將依法令規定公告相關事項。
2,1101 台泥,2018-08-21,001A001,3(2018/03),I_產業前景,IF01_增資/減資/融資發債,馬來西亞商POWER INFRASTRUCTURE HOLDINGS(MALAYSIA)SD...
3,1101 台泥,2018-08-21,001A002,3(2018/03),I_產業前景,IF01_增資/減資/融資發債,電力股份有限公司，經營火力發電廠等業務。
4,1101 台泥,2018-08-21,001B001,3(2018/03),I_產業前景,IF01_增資/減資/融資發債,【TCRI 觀點_20180827】


In [53]:
data = data[['公司', '事件日', '小事件類別', '事件內容']]
data.head()

,公司,事件日,小事件類別,事件內容
0,1101 台泥,2018-09-14,IF02_投資/併購/組織策略,1.本公司與土耳其SANKO Holding A.S.公司就其下水泥事業群之股權投資事宜簽署...
1,1101 台泥,2018-09-14,IF02_投資/併購/組織策略,2.有關具體交易條件，待後續雙方正式簽署拘束性合約時，將依法令規定公告相關事項。
2,1101 台泥,2018-08-21,IF01_增資/減資/融資發債,馬來西亞商POWER INFRASTRUCTURE HOLDINGS(MALAYSIA)SD...
3,1101 台泥,2018-08-21,IF01_增資/減資/融資發債,電力股份有限公司，經營火力發電廠等業務。
4,1101 台泥,2018-08-21,IF01_增資/減資/融資發債,【TCRI 觀點_20180827】


In [54]:
data.sort_values(by = ['公司', '事件日', '小事件類別'], inplace = True)
data.reset_index(inplace = True)
data.drop('index', axis = 1, inplace = True)
data.head()

,公司,事件日,小事件類別,事件內容
0,1101 台泥,2016-01-08,MC03_工安問題,1.公告本公司花蓮廠火災對財務、業務無重大影響
1,1101 台泥,2016-01-08,MC03_工安問題,2.本公司花蓮廠廠內原有報廢的三台重油發電機出售予(建价公司)，該公司於今
2,1101 台泥,2016-01-08,MC03_工安問題,日上午拆卸重油發電機時發生火星掉入下方的電纜溝槽引起電纜起火造成此次
3,1101 台泥,2016-01-08,MC03_工安問題,事故，經消防隊全力搶救，已確認火勢全部撲滅，沒有人員傷亡。
4,1101 台泥,2016-01-08,MC03_工安問題,3.(1)本公司廠房及設備均足額投保，已通知保險公司，待勘查結果進行理賠程


In [71]:
group = data.groupby(by = ['公司', '事件日', '小事件類別'])
company = []
date = []
category = []
detail = []
for i in group.groups:
    company.append(i[0])
    date.append(i[1].date())
    category.append(i[2])
    text = list(map(str, group.get_group(i)['事件內容'].tolist()))
    detail.append(' '.join(text))

In [72]:
combine = pd.DataFrame({'公司' : company,
                        '事件日' : date, 
                        '小事件類別' : category,
                        '事件內容' : detail})[['公司', '事件日', '小事件類別', '事件內容']]
combine.sort_values(by = ['公司', '事件日', '小事件類別', '事件內容'], inplace=True)


In [73]:
combine.head()

,公司,事件日,小事件類別,事件內容
0,1101 台泥,2016-01-08,MC03_工安問題,1.公告本公司花蓮廠火災對財務、業務無重大影響 2.本公司花蓮廠廠內原有報廢的三台重油發電機...
1,1101 台泥,2016-01-22,MO06_董監持股質押,"董事本人清園投資2016/01/22設質1,000,000股給大中票券金融，累計設質2,42..."
2,1101 台泥,2016-02-01,MT02_董監異動,改派1董 董事張榮(信和投資代表)卸任 董事張啟文(信和投資代表)新任
3,1101 台泥,2016-02-05,IS01_營收/ 接單/ 客戶變化,"2016年01月累計營收6,575,590千元，年減26%。2016年01月單月營收6,57..."
4,1101 台泥,2016-02-26,MO01_董監/大股東持股申讓,"法人董事代表人本人辜成允於2016/02/26預計申讓10,000張股票，預計申讓比重31%..."


In [75]:
# combine.to_excel('./1106_看門狗_全產業分類整理-1test.xlsx', index = False)

In [67]:
df = combine
df.head()

,公司,事件日,小事件類別,事件內容
0,1101 台泥,2016-01-08,MC03_工安問題,1.公告本公司花蓮廠火災對財務、業務無重大影響2.本公司花蓮廠廠內原有報廢的三台重油發電機出...
1,1101 台泥,2016-01-22,MO06_董監持股質押,"董事本人清園投資2016/01/22設質1,000,000股給大中票券金融，累計設質2,42..."
2,1101 台泥,2016-02-01,MT02_董監異動,改派1董董事張榮(信和投資代表)卸任董事張啟文(信和投資代表)新任
3,1101 台泥,2016-02-05,IS01_營收/ 接單/ 客戶變化,"2016年01月累計營收6,575,590千元，年減26%。2016年01月單月營收6,57..."
4,1101 台泥,2016-02-26,MO01_董監/大股東持股申讓,"法人董事代表人本人辜成允於2016/02/26預計申讓10,000張股票，預計申讓比重31%..."


In [68]:
df_1101 = df[df.公司 == '1101 台泥']

In [69]:
df_1101

,公司,事件日,小事件類別,事件內容
0,1101 台泥,2016-01-08,MC03_工安問題,1.公告本公司花蓮廠火災對財務、業務無重大影響2.本公司花蓮廠廠內原有報廢的三台重油發電機出...
1,1101 台泥,2016-01-22,MO06_董監持股質押,"董事本人清園投資2016/01/22設質1,000,000股給大中票券金融，累計設質2,42..."
2,1101 台泥,2016-02-01,MT02_董監異動,改派1董董事張榮(信和投資代表)卸任董事張啟文(信和投資代表)新任
3,1101 台泥,2016-02-05,IS01_營收/ 接單/ 客戶變化,"2016年01月累計營收6,575,590千元，年減26%。2016年01月單月營收6,57..."
4,1101 台泥,2016-02-26,MO01_董監/大股東持股申讓,"法人董事代表人本人辜成允於2016/02/26預計申讓10,000張股票，預計申讓比重31%..."
5,1101 台泥,2016-03-10,IS01_營收/ 接單/ 客戶變化,"2016年02月累計營收10,207,643千元，年減26%。2016年02月單月營收3,6..."
6,1101 台泥,2016-04-22,MC02_環保問題,1.代子公司和平電力股份有限公司公告接獲最高行政法院書記科通知。2.本公司子公司和平電力今(...
7,1101 台泥,2016-06-22,MT02_董監異動,補選4董董事謝其嘉(富品投資代表)新任董事陳啟德(恆強投資代表)新任董事麥維德(辜公亮文教基...
8,1101 台泥,2016-09-10,RB01_TCRI負向觀察,2016/09/10 依2016/06財報新增向下觀察，原因為：持續觀察中國水泥市場狀況
9,1101 台泥,2016-11-22,MO06_董監持股質押,"董事本人國產建材實業2016/11/22設質2,500,000股給兆豐國際商業銀行，累計設質..."


In [ ]:
def commaparser(string):
    for i in string:
        

In [70]:
for row in df_1101.iloc[:,3]:
    seg = jieba.cut(row, cut_all = False)    #jieba分词
    print('/'.join(seg))


1/./公告/本/公司/花蓮廠/火災/對/財務/、/業務/無/重大/影響/2/./本/公司/花蓮廠/廠/內/原有/報廢/的/三台/重油/發電機/出售/予/(/建价/公司/)/，/該/公司/於/今日/上午/拆卸/重油/發電機/時/發生/火星/掉入/下方/的/電纜/溝槽/引起/電纜/起火/造成/此次/事故/，/經/消防/隊/全力/搶救/，/已確/認火勢/全部/撲滅/，/沒/有人/員/傷亡/。/3/./(/1/)/本/公司/廠房/及/設備/均/足額/投保/，/已/通知/保險/公司/，/待/勘查/結果/進行理/賠/程序/。/(/2/)/公司/財務業務/影響/:/在/完全/復/工前/，/將由/其他/廠區/支援/生產/，/對/公司/財務/、/業務/無/重大/影響/。
董事/本人/清園/投資/2016///01///22/設質/1/,/000/,/000/股給/大中/票券/金融/，/累計/設質/2/,/420/,/000/股/。
改派/1/董/董事/張榮/(/信/和/投資/代表/)/卸任/董事/張/啟文/(/信/和/投資/代表/)/新任
2016/年/01/月/累計/營收/6/,/575/,/590/千元/，/年減/26%/。/2016/年/01/月/單月/營收/6/,/575/,/590/千元/，/年減/26%/。
法人/董事/代表人/本人/辜成/允/於/2016///02///26/預計/申讓/10/,/000/張/股票/，/預計/申讓/比重/31%/，/交易方式/為鉅額/逐筆/交易/。
2016/年/02/月/累計/營收/10/,/207/,/643/千元/，/年減/26%/。/2016/年/02/月/單月/營收/3/,/632/,/053/千元/，/年減/25%/。
1/./代/子公司/和平/電力/股份/有限公司/公告/接獲/最高/行政法院/書記科/通知/。/2/./本/公司/子公司/和平/電力/今/(/4///22/)/獲悉/「/和平/電力/98/、/99/年度/用/煤/超量/行政/裁罰案/」/業經/最高/行政法院/駁回/上訴/，/判決確/定金/額為/新/台幣/4/億/3/千/6/百餘萬元/ /。/上述/裁罰/金額業/已/於/101/年度/估列/相關/損失/，/且/已/於/103/年度/全數/繳納/上述/罰/鍰/，/待/收受/最高/行政法院/判決/後/，/與律師/研議後